# Model training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import sys
import gc
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
import time as time_m
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import tree

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import  FeatureUnion #Pipeline,
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
# from sklearn.externals import joblib
from datetime import datetime
from sklearn import metrics

In [ ]:
!ls

drive  sample_data


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS5500/dataset/feature_selected_train&test.csv')
cate = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS5500/dataset/feature_selected_train&test_cate_woe.csv')

In [ ]:
cols = cate.columns
data[cols] = cate[cols]

In [ ]:
cols = []
with open('/content/drive/MyDrive/Colab Notebooks/DS5500/feature_names.txt') as f:
    for line in f.readlines():
        cols.append(line.strip()) 
# print(cols)

In [ ]:
train = data[(data['data']=='Train') & (data['target'] != -1)]
test = data[(data['data']=='Test') & (data['target'] != -1)]
train = train.drop('data',axis=1)
test = test.drop('data',axis=1)

In [ ]:
del data, cate
gc.collect()

363

In [ ]:
train_x = train[cols]
test_x = test[cols]
# columns_to_drop = list(train_x.columns[:6])
# train_x = train_x.drop(columns_to_drop,axis = 1)
# test_x = test_x.drop(columns_to_drop, axis = 1)
train_y = train.target.astype("int")

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(train_x,train_y, test_size=0.3,random_state=5,stratify=train_y)

In [ ]:
train_x.shape

(729606, 85)

In [ ]:
train_y.value_counts()

0    684530
1     45076
Name: target, dtype: int64

In [ ]:
684530/45076

15.18613009140119

In [ ]:
train_x = train_x.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_x = test_x.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
del train,test
gc.collect()

0

In [ ]:
# tuned models
lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=20, metric='auc',
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_iterations=150, num_leaves=35,
                                objective='binary', random_state=None,
                                reg_alpha=0.0, reg_lambda=0.0, seed=114514,
                                silent=True, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0)

rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=15, n_jobs=None,
                                        oob_score=False, random_state=114514,
                                        verbose=0, warm_start=False)

xgb = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=0.7, colsample_bynode=1,
                               colsample_bytree=0.7, eval_metric='auc', gamma=1,
                               learning_rate=0.1, max_delta_step=0, max_depth=5,
                               min_child_weight=1.1, missing=None,
                               n_estimators=150, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=15,
                               seed=0, silent=None, subsample=0.7,
                               tree_method='hist', verbosity=1)

lr = LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=-1, penalty='l2',
                                    random_state=114514, solver='saga',
                                    tol=0.0001, verbose=0, warm_start=False)

simpleImputer = SimpleImputer(strategy='most_frequent')

In [ ]:
# svm = SVC(probability=True) #class_weight=svmweight, C=svmc, kernel=svmkernel, 
rf = RandomForestClassifier(random_state=114514)
lr = LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=-1, penalty='l2',
                                      random_state=114514, solver='saga',
                                      tol=0.0001, verbose=0, warm_start=False)
xgb = XGBClassifier(gamma=1,seed=114514)
smote = SMOTE(random_state=114514)
ros = RandomOverSampler(sampling_strategy = 'auto',random_state=114514)
lgb = LGBMClassifier(n_jobs=-1)
simpleImputer = SimpleImputer(strategy='most_frequent')

In [ ]:
def get_param_grid(clr):
    if clr == 'lgb':
        param_grid = dict(
            classifier__metric = ['auc', 'binary_logloss'],
            classifier__num_iterations=[50,100,150],
            classifier__seed = [114514],
            classifier__max_depth = [10, 20],
            classifier__objective = ['binary'],
            upsample__sampling_strategy = [0.1,0.7],
            classifier__num_leaves = [15,35] #be smaller than 2^(max_depth)
        )
    elif clr == 'rf':
        param_grid = dict(
            classifier__n_estimators = [10,15],
            upsample__sampling_strategy = [0.1,0.7]
        )
    elif clr == 'lr':
        param_grid = dict(
            classifier__penalty = ['none','l1','l2'],
            classifier__class_weight = ['balanced'],
            upsample__sampling_strategy = [0.1,0.7]
        )
    elif clr == 'xgb':
        param_grid = dict(
            classifier__booster = ['gbtree'],
            classifier__n_estimators=[50,100,150],
            classifier__learning_rate=[0.1],
            classifier__max_depth=[5],
            classifier__tree_method = ['hist'],
            classifier__objective = ['binary:logistic'],
            classifier__eval_metric = ['auc'],
            classifier__min_child_weight = [1.1],
            classifier__seed = [0],
            classifier__subsample = [0.7],
            classifier__colsample_bytree = [0.7],
            classifier__colsample_bylevel = [0.7],
            classifier__scale_pos_weight = [15]
        )
    return param_grid
        

In [ ]:
models = []
# for classifier in zip([lgb,lr,rf,xgb],['lgb','lr','rf','xgb']):
for classifier in zip([rf,lgb,xgb],['rf','lgb','xgb']):
    print("Performing grid search...")
    print("Classifier:", [classifier[1]])
    # param_grid = get_param_grid(classifier[1])
    pipeline = Pipeline([('classifier',classifier[0])])
    # grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10, n_jobs=-1, cv= 3,scoring='roc_auc')
    grid_search = GridSearchCV(pipeline, param_grid={}, verbose=10, n_jobs=-1, cv= 3,scoring='roc_auc')
    if classifier[0] == xgb or classifier[0] == lgb:
        grid_search.fit(train_x, train_y)
        prob = grid_search.predict_proba(test_x)
    else:
        cache = train_x.fillna(0)
        grid_search.fit(cache, train_y)
        cache = test_x.fillna(0)
        prob = grid_search.predict_proba(cache)
        del cache
        gc.collect()
    print("test_auc : ",metrics.roc_auc_score(test_y,prob[:,1]))
    models.append((grid_search.best_score_,grid_search.best_estimator_))

Performing grid search...
Classifier: ['rf']
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   43.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   43.0s finished


test_auc :  0.7077654903812336
Performing grid search...
Classifier: ['lgb']
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.5min finished
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


test_auc :  0.7437949722566107
Performing grid search...
Classifier: ['xgb']
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.5min finished


test_auc :  0.7411904469100824


In [ ]:
models_restore = models

In [ ]:
lr_pipeline = Pipeline([('impute',simpleImputer),('scale',StandardScaler()),('classifier',lr)])
grid_search = GridSearchCV(lr_pipeline, param_grid={}, verbose=10, n_jobs=-1, cv= 3,scoring='roc_auc')
grid_search.fit(train_x, train_y)
models.append((grid_search.best_score_,grid_search.best_estimator_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.1min finished
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, 
                    max_iter=300, 
                    hidden_layer_sizes = 400,
                    learning_rate = 'invscaling')

In [ ]:
mlp_pipeline = Pipeline([('impute',simpleImputer),('scale',StandardScaler()),('classifier',lr)])
grid_search = GridSearchCV(mlp_pipeline, param_grid={}, verbose=10, n_jobs=-1, cv= 3,scoring='roc_auc')
grid_search.fit(train_x, train_y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.2min finished
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('impute',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='most_frequent',
                                                      verbose=0)),
                                       ('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('classifier',
                                        LogisticRegression(C=1.0,
                                                           

In [ ]:
grid_search.best_score_

0.7101664135226023

In [ ]:
prob = grid_search.predict_proba(test_x)
metrics.roc_auc_score(test_y,prob[:,1])

0.709681322780316

In [ ]:
# models_restore[0][1].get_params()

In [ ]:
def get_predict(models, train_x, train_y, test_x):
    total_score = sum([i[0] for i in models])
    res_train = []
    res_test = []
    models_trained = []
    for i in models:
        if 'xgb' in str(i[1]) or 'lgb' in str(i[1]):
            i[1].fit(train_x, train_y)
            y = i[1].predict_proba(test_x)
            y = y*i[0]/total_score
            res_test.append(y)
            y = i[1].predict_proba(train_x)
            y = y*i[0]/total_score
            res_train.append(y)
            models_trained.append(i[1])
        else:
            cache = train_x.fillna(0)
            i[1].fit(cache, train_y)
            y = i[1].predict_proba(cache)
            y = y*i[0]/total_score
            res_train.append(y)
            cache = test_x.fillna(0)
            y = i[1].predict_proba(cache)
            y = y*i[0]/total_score
            res_test.append(y)
            models_trained.append(i[1])
            del cache
            gc.collect()
    return res_train, res_test, models_trained

In [ ]:
res_train, res_test,models_trained = get_predict(models, train_x, train_y, test_x)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
predict_prob_test = [i[1] for i in sum(res_test)]
predict_prob_train = [i[1] for i in sum(res_train)]

In [ ]:
## weighted average
print("Train set:", metrics.roc_auc_score(train_y,predict_prob_train) )
print("Test set:",metrics.roc_auc_score(test_y,predict_prob_test))

Train set: 0.7862863709788229
Test set: 0.7397274757209471


In [ ]:
## model training stack
model_learn_test = []
for i in range(len(res_test[0])):
  cache = []
  for j in range(4):
    cache.append(res_test[j][i][0])
  model_learn_test.append(cache)

In [ ]:
model_learn_train = []
for i in range(len(res_train[0])):
  cache = []
  for j in range(4):
    cache.append(res_train[j][i][0])
  model_learn_train.append(cache)

In [ ]:
res_train[0][0][0]

0.23178979926754847

In [ ]:
model_learn_train = np.array(model_learn_train)
model_learn_test = np.array(model_learn_test)

In [ ]:
lr_model  = LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=10,
                                    multi_class='auto', n_jobs=-1, penalty='l1',
                                    random_state=114514, solver='saga',
                                    tol=0.0001, verbose=0, warm_start=False)

In [ ]:
lr_model.fit(model_learn_train,train_y)
y_test = lr_model.predict_proba(model_learn_test)
y_train = lr_model.predict_proba(model_learn_train)
print("test :",metrics.roc_auc_score(test_y,y_test[:,1]))
print("train :",metrics.roc_auc_score(train_y,y_train[:,1]))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


test : 0.6985957768024339
train : 0.8270443948023503
